In [1]:
import sys
import warnings
import pandas as pd
import numpy as np
import os
import cx_Oracle as O
import datetime as dt
import logging, logging.handlers, smtplib
import traceback
import smtplib
import sqlalchemy as sa
import pyodbc
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from os.path import basename
import os
from datetime import datetime, date
import settings
import errhandler as eh
import eHSQL
import re
import os

global df, dfQA
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

con = None
try:
    global conn, recordlist, df, dfQA
    conn = False
    logging.basicConfig(filename = 'eHARSExtract.log',
                       filemode='a',
                       format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                       datefmt='%H:%M:%S',
                       level=logging.DEBUG)
    logging.info(f'\nLOG START: {datetime.now()}\n')

    #Opens connection to the Oracle DB via the stored config file in the config folder
    def open_connection():
        global driver, database, uid, pwd, connection, con
        #connection = 'DRIVER={Oracle in instantclient_21_6};DBQ=NDEDSS_DEVsvc.itd.nd.gov;UID=NDDH_EDSS_Intr;PWD=QRSTUVW1'
        connection = f'{settings.username}/{settings.password}@{settings.servicename}'
        con = O.connect(connection)
        #con = pyodbc.connect(connection)
        return con

    def connection_test():
        cur = con.cursor()
        cur.execute("SELECT * FROM v$version")
        row = cur.fetchone()
        print("Connection established to: ", row[0])
        cur.close()
        con.commit()
        logging.info(f"\nConnection established to: {row[0]}\n")
        return

    def close_connection():
        con.close()
        return

    def tablecount(con):
        cur = con.cursor()
        cur.execute(eHSQL.SQLCaseCount)
        output = cur.fetchone()
        if output[0] == 0:
            print(f'\n{output[0]} Existing Records\n', end = "\r")
            logging.info(f'\n{output[0]} Existing records\n')
            cur.close()
            con.commit()
            return False
        else:
            print(f'\n{output[0]} Existing Records\n', end = "\r")
            logging.info(f'\n{output[0]} Existing records\n')
            cur.close()
            con.commit()
            return True

    def tablecontent(con):
        global recordlist
        cur = con.cursor()
        SQL = eHSQL.SQLHIVData
        cur.execute(SQL)
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            df = pd.read_sql(SQL, con)
            display(df)
        output = cur.fetchall()
        output = output[0:]
        output = str(tuple(list(list(zip(*output))[0])))
        #print(output)
        #print(f'\nRecord List: {output}\n', end = "\r")
        logging.info(f'\nRecord List: {output}\n')
        recordlist = output
        cur.close()
        con.commit()
        return

    def get_data(con, recordlist):
        global df
        cur = con.cursor()
        print(recordlist)
        gr1 = "'Test'"
        gr2 = "'LabFacility'"
        gr3 = "'OrderingProvider'"
        gr4 = "'SpecimenInfo'"
        gr5 = "'Misc'"
        SQLtest = f'{eHSQL.SQLLabFields}' + 'WHERE cs.case_id in ' + recordlist + f' and INVResult.Result_Code = {gr1}' + ' ORDER BY cs.case_id, INVResultAttr.Name'
        SQLLF = f'{eHSQL.SQLLabFields}' + 'WHERE cs.case_id in ' + recordlist + f' and INVResult.Result_Code = {gr2}' + ' ORDER BY cs.case_id, INVResultAttr.Name'
        SQLOP = f'{eHSQL.SQLLabFields}' + 'WHERE cs.case_id in ' + recordlist + f' and INVResult.Result_Code = {gr3}' + ' ORDER BY cs.case_id, INVResultAttr.Name'
        SQLSI = f'{eHSQL.SQLLabFields}' + 'WHERE cs.case_id in ' + recordlist + f' and INVResult.Result_Code = {gr4}' + ' ORDER BY cs.case_id, INVResultAttr.Name'
        SQLM = f'{eHSQL.SQLLabFields}' + 'WHERE cs.case_id in ' + recordlist + f' and INVResult.Result_Code = {gr5}' + ' ORDER BY cs.case_id, INVResultAttr.Name'

        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            warnings.simplefilter('ignore', pd.errors.PerformanceWarning)
            warnings.simplefilter('ignore', FutureWarning)
            
            cur.execute(SQLtest)
            dfTest = pd.read_sql(SQLtest, con)
            
            cur.execute(SQLLF)
            dfLF = pd.read_sql(SQLLF, con)
            
            cur.execute(SQLOP)
            dfOP = pd.read_sql(SQLOP, con)

            cur.execute(SQLSI)
            dfSI = pd.read_sql(SQLSI, con)
            
            cur.execute(SQLM)
            dfM = pd.read_sql(SQLM, con)
            
            df_test = dfTest.pivot(index = ['INVRESULTUNID','INVUNID','CASE_ID'], columns = ['NAME'], values=['VALUE'])
            df_test.columns = df_test.columns.droplevel(0)
            df_test = df_test.rename_axis(None, axis = 1).reset_index()
            
            df_LF = dfLF.pivot(index = ['INVRESULTUNID','INVUNID','CASE_ID'], columns = ['NAME'], values=['VALUE'])
            df_LF.columns = df_LF.columns.droplevel(0)
            df_LF = df_LF.rename_axis(None, axis = 1).reset_index()
            df_LF = df_LF.drop(columns = ['INVRESULTUNID','CASE_ID','Facility','Facility_Other'])
            
            df_OP = dfOP.pivot(index = ['INVRESULTUNID','INVUNID','CASE_ID'], columns = ['NAME'], values=['VALUE'])
            df_OP.columns = df_OP.columns.droplevel(0)
            df_OP = df_OP.rename_axis(None, axis = 1).reset_index()
            df_OP = df_OP.drop(columns = ['INVRESULTUNID','CASE_ID','Address', 'City', 'Name', 'PostalCode', 'State'])
            
            df_SI = dfSI.pivot(index = ['INVRESULTUNID','INVUNID','CASE_ID'], columns = ['NAME'], values=['VALUE'])
            df_SI.columns = df_SI.columns.droplevel(0)
            df_SI = df_SI.rename_axis(None, axis = 1).reset_index()
            df_SI = df_SI.drop(columns = ['INVRESULTUNID','CASE_ID'])
            
            df_M = dfM.pivot(index = ['INVRESULTUNID','INVUNID','CASE_ID'], columns = ['NAME'], values=['VALUE'])
            df_M.columns = df_M.columns.droplevel(0)
            df_M = df_M.rename_axis(None, axis = 1).reset_index()
            df_M = df_M.drop(columns = ['INVRESULTUNID','CASE_ID'])
            
            df = dfTest.drop(columns = ['NAME','VALUE','CASE_ID','INVUNID','RESULT_CODE']).drop_duplicates()

            df = pd.merge(df, df_test, how ='outer', on = 'INVRESULTUNID')

            df['record_no'] = np.arange(len(df))+1
            rcount = df.pop('record_no')
            df.insert(0, rcount.name, rcount)
            
            #df.loc[df['INVUNID'].isna(), 'INVUNID'] = df['record_no']
            
            df.sort_values(by=['INVRESULTUNID'])
            #display(df)
            df['INVUNID'] = df['INVUNID'].astype('str')
            df['binary'] = df['INVUNID'].shift(1, fill_value = df['INVUNID'].head(1)) != df['INVUNID']
            df['parent_id'] = df['INVUNID'].groupby((df['binary']).cumsum()).cumcount()+1
            df['U_ID'] = df.groupby('INVUNID', sort=False).ngroup()+1
            df['rollup_no'] = np.nan
            df.loc[df['parent_id'] == 1, 'rollup_no'] = df['record_no']
            df['rollup_no'] = df['rollup_no'].ffill()
            df['rollup_no'] = df['rollup_no'].astype('int')
            
            df.set_index('record_no', inplace = True)

            df = df.drop(columns = ['binary'])
            
            col = df.pop('U_ID')
            df.insert(1, col.name, col)
            
            col = df.pop('parent_id')
            df.insert(2, col.name, col)
            
            col = df.pop('rollup_no')
            df.insert(3, col.name, col)
            
            #############
            
            df_LF['record_no'] = np.arange(len(df_LF))+1
            rcount = df_LF.pop('record_no')
            df_LF.insert(0, rcount.name, rcount)

            df_LF['INVUNID'] = df_LF['INVUNID'].astype('str')
            df_LF['binary'] = df_LF['INVUNID'].shift(1, fill_value = df_LF['INVUNID'].head(1)) != df_LF['INVUNID']
            df_LF['parent_id'] = df_LF['INVUNID'].groupby((df_LF['binary']).cumsum()).cumcount()+1
            df_LF = df_LF.drop(columns = ['record_no','binary'])
            #display(df_LF)
            
            #############
            
            df_OP['record_no'] = np.arange(len(df_OP))+1
            rcount = df_OP.pop('record_no')
            df_OP.insert(0, rcount.name, rcount)

            df_OP['INVUNID'] = df_OP['INVUNID'].astype('str')
            df_OP['binary'] = df_OP['INVUNID'].shift(1, fill_value = df_OP['INVUNID'].head(1)) != df_OP['INVUNID']
            df_OP['parent_id'] = df_OP['INVUNID'].groupby((df_OP['binary']).cumsum()).cumcount()+1
            df_OP = df_OP.drop(columns = ['record_no','binary'])
            #display(df_OP)
            
            ##############
            
            df_SI['record_no'] = np.arange(len(df_SI))+1
            rcount = df_SI.pop('record_no')
            df_SI.insert(0, rcount.name, rcount)

            df_SI['INVUNID'] = df_SI['INVUNID'].astype('str')
            df_SI['binary'] = df_SI['INVUNID'].shift(1, fill_value = df_SI['INVUNID'].head(1)) != df_SI['INVUNID']
            df_SI['parent_id'] = df_SI['INVUNID'].groupby((df_SI['binary']).cumsum()).cumcount()+1
            df_SI = df_SI.drop(columns = ['record_no','binary'])
            #display(df_SI)
            
            ##############
            
            df_M['record_no'] = np.arange(len(df_M))+1
            rcount = df_M.pop('record_no')
            df_M.insert(0, rcount.name, rcount)

            df_M['INVUNID'] = df_M['INVUNID'].astype('str')
            df_M['binary'] = df_M['INVUNID'].shift(1, fill_value = df_M['INVUNID'].head(1)) != df_M['INVUNID']
            df_M['parent_id'] = df_M['INVUNID'].groupby((df_M['binary']).cumsum()).cumcount()+1
            df_M = df_M.drop(columns = ['record_no','binary'])
            #display(df_M)
            
            ##############
            
            df = pd.merge(df, df_LF, how ='outer', on = ['INVUNID','parent_id'])
            df = pd.merge(df, df_OP, how ='outer', on = ['INVUNID','parent_id'])
            df = pd.merge(df, df_SI, how ='outer', on = ['INVUNID','parent_id'])
            df = pd.merge(df, df_M, how ='outer', on = ['INVUNID','parent_id'])
            
            #display(df)
            
            recordadded = False
            isGenoType = False
            genotypeid = ['61199-6','34700-5','45176-5','45175-7','33630-5','53798-5','90903-6','81122-4']
            
            df['doc_type'] = "004"
            df['source'] = "A05.03"
            df['surv_method'] = "P"
            
            datelist = ('CREATE_DATE', 'MODIFICATION_DATE', 'BIRTH_DATE')
            date_format = "%m/%d/%Y"
            
            for date in enumerate(datelist):
                if date[1] in df:
                    df[date[1]] = df[date[1]].dt.strftime(date_format)
                else:
                    continue
                    
            df['enter_dt'] = df['CREATE_DATE']
            df['rcvd_dt'] = df['CREATE_DATE']
            df['m_name'] = ''
            df.loc[df['parent_id'] == 1, 'f_name'] = df['FIRST_NAME']
            df.loc[df['parent_id'] == 1, 'l_name'] = df['LAST_NAME']
            df.loc[df['parent_id'] == 1, 'birth_dt'] = df['BIRTH_DATE']
            df.loc[df['parent_id'] == 1, 'name_use_cd'] = 'L'
            df['isGenoType'] = False
            df['specimen_id'] = df['SpecimenNumber']
            df['lab_specific_note'] = ''
            df['comments'] = ''
            df['notes'] = ''
            df['name_use_cd'] = ''
            df['addr_type_cd'] = ''
            df['facility_name'] = df['Facility']
            df['facility_ctry'] = 'USA'
            df['facility_phone'] = df['Phone']
            df['refer_obgyn'] = ''
            
            try:
                df['notes'] = df['Notes']
            except KeyError:
                df['notes'] = ''
                pass
            try:
                df['lab_clia_no'] = df['CLIA']
            except KeyError:
                df['lab_clia_no'] = ''
                pass
            try:
                df['accession_no'] = df['SpecimenNumber']
            except KeyError:
                df['accession_no'] = ''
                pass
            try:
                df['colldate'] = df['SpecimenDate']
            except KeyError:
                df['colldate'] = ''
                pass
                
            df['result_dt'] = df['SpecimenDate']
            df['specimen_recieve_dt'] = df['SpecimenReceivedDate']
            df['specimentype'] = df['SpecimenSource']
            df['lab_result'] = ''
            df['lab_units'] = ''
            
            df['lab_test_name'] = df['Test']
            df['genotype_sequence'] = ''
            df['starhs_sample_id'] = ''
            df['sreason'] = ''
                
            #Interpretation condition values
            interpret1 = ['20447-9','25836-8','29541-0','24013-5','25835-0','59419-2','70241-5']
            statuslist = ['24013-5','29541-0','25835-0']
            interpret2 = ['29541-0','24013-5']

            PCT = ['8128-1','8123-2','32516-7']
            CNT = ['8127-3','24467-3','32515-9']
            CML = ['20447-9','25836-8','24013-5','59419-2','70241-5','25835-0']
            LOG = ['29541-0']
            
            units = ['PCT','CNT','Copies/ML','Log']
            
            POS = ['10828004','G-A200','11214006','G-A497','261420005','260373001']
            NEG = ['260415000','260385009']
            IND = ['82334004']
            
            result = ['POS', 'NEG', 'IND']
            
            #Identify HIV/STD cases
            for gid in enumerate(genotypeid):
                df.loc[df['Test'] == str(gid[1]), 'isGenoType'] = True
            
            #Identify UOM (Unit of Measurement)
            for vals in enumerate(PCT):
                df.loc[df['Test'] == str(vals[1]), 'lab_units'] = str(units[0])
            
            for vals in enumerate(CNT):
                df.loc[df['Test'] == str(vals[1]), 'lab_units'] = str(units[1])
              
            for vals in enumerate(CML):
                df.loc[df['Test'] == str(vals[1]), 'lab_units'] = str(units[2])
            
            for vals in enumerate(LOG):
                df.loc[df['Test'] == str(vals[1]), 'lab_units'] = str(units[3])
                
            df.loc[(df['Comments'] != 'None') | (df['notes'] != ''),'lab_specific_note'] = df['Comments'][0:100]
            df.loc[(df['Comments'] != 'None') | (df['notes'] != ''),'comments'] = df['Comments'][0:255]
            
            df.loc[(df['ResultValue'].str.contains('ND00')) & (df['isGenoType'] == True),'genotype_sequence'] = df['EXTENDED_VALUE']
            df.loc[(df['ResultValue'].str.contains('ND00')) & (df['isGenoType'] == True),'lab_units'] = 'BP'
            
            df.loc[(df['isGenoType'] == True), 'genotype_sequence'] = df['EXTENDED_VALUE']
            df.loc[(df['isGenoType'] == True),'lab_units'] = 'BP'
            
            #Identify UOM (Unit of Measurement)
            for vals in enumerate(POS):
                df.loc[df['Result'] == str(vals[1]), 'lab_result'] = str(result[0])
            for vals in enumerate(NEG):
                df.loc[df['Result'] == str(vals[1]), 'lab_result'] = str(result[1])
            for vals in enumerate(IND):
                df.loc[df['Result'] == str(vals[1]), 'lab_result'] = str(result[2])
                    
            if df['ResultValue'].iteritems() != 'None':
                if df['ResultValue'].str.contains(';').items():
                    df['ResultValue'] = df['ResultValue'].str.replace('^[^;]*;','').str.replace('[A-Z]','')
            
            if df['ResultValue'].iteritems() != 'None':
                if df['ResultValue'].str.contains('<').items():
                    df['ResultValueTemp'] = df['ResultValue'].str.replace("<","")
                    
            df.loc[(df['lab_result'] == 'None') & (df['isGenoType'] == False), 'lab_result'] = df['CASE_ID']
            df.loc[(df['lab_result'] == '') ,'lab_result'] = df['ResultValueTemp']

            for vals in enumerate(interpret1):
                df.loc[(df['Test'] == str(vals[1])) & (df['ResultValue'].str.contains('<')), 'lab_result'] = '0'
                df.loc[(df['Test'] == str(vals[1])) & (df['ResultValue'].str.contains('<')), 'interpretation'] = '<'
                
                df['result_range_upper'] = 10000000
                df.loc[(df['Test'] == str(vals[1])) & (df['ResultValue'].str.contains('<')) & ('.' not in df['lab_result']), 'result_range_lower'] = df['lab_result']
                
            for vals in enumerate(statuslist):
                df.loc[(df['Test'] == str(vals[1])) & (df['ReferenceRange'].str.contains('DETECTED')),'lab_result'] = '0'
                df.loc[(df['Test'] == str(vals[1])) & (df['ReferenceRange'].str.contains('NOT DETECTED')),'lab_result'] = None
                df.loc[(df['Test'] == str(vals[1])) & (df['ReferenceRange'].str.contains('NOT QUANTIFIED')),'lab_result'] = None
            
            for vals in enumerate(interpret2):
                df.loc[(df['Test'] == str(vals[1])) & (df['ResultValueTemp'].iteritems() != 'None'),'interpretation'] = '<'
                df.loc[(df['Test'] == str(vals[1])) & (df['ResultValueTemp'].iteritems() != 'None') & (df['ResultValueTemp'] >= '0'),'interpretation'] = '='
            
            df['result_range_lower'] = 20
            df.loc[df['Test'] == '59419-2', 'result_range_lower'] = 40
            df['result_range_upper'] = 10000000
            
            df.loc[(df['ResultValueTemp'] != 'None') & (df['ResultValueTemp'] < '20'), 'interpretation'] = '<'
            df.loc[((df['ResultValueTemp'] != 'None') & (df['ResultValueTemp'] >= '20') & (df['ResultValueTemp'] <= '10000000')), 'interpretation'] = '='
            df.loc[(df['ResultValueTemp'] != 'None') & (df['ResultValueTemp'] > '10000000'), 'interpretation'] = '>'
            
            df.loc[df['result_range_lower'] == '1.6', 'result_range_lower'] = '' 
            df.loc[df['lab_test_name'] == '20447-9', 'lab_test_name'] = '59419'            
            df.loc[((df['lab_test_name'] == '20447-9') | (df['lab_test_name'] == '25836-8') | (df['lab_test_name'] == '29541-0') | (df['lab_test_name'] == '70241-5') | (df['lab_test_name'] == '25835-0')) , 'lab_test_name'] = '74854'        
            df.loc[((df['lab_test_name'] == '24467-3') | (df['lab_test_name'] == '32515-9')) , 'lab_test_name'] = '24467'  
            df.loc[((df['lab_test_name'] == '8123-2') | (df['lab_test_name'] == '32516-7')) , 'lab_test_name'] = '32516-7'
            df['lab_result'] = df['lab_result'].str.replace('=','')
            
            #display(df)
    
            with pd.option_context('display.max_rows', None, 'display.max_columns', None):
                display(df)
            
        output = cur.fetchall()
        logging.info(f'\nData Succesfully Prepped.\n')
        cur.close()
        con.commit()
        return df
    
    def get_QandA(con, recordlist):
        cur = con.cursor()
        SQL = f'{eHSQL.SQLQandA}' + recordlist
        cur.execute(SQL)
        global dfQA
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            #warnings.simplefilter('ignore', UserWarning)
            dfQA = pd.read_sql(SQL, con)
            
            QSIDList = ['ADMINISTR','DEMOGRAPHIC','RISK','CLINICAL']
            
            dfQA = dfQA.loc[dfQA['QUESTIONSET_ID'].str.contains('|'.join(QSIDList))]
            df_test = dfQA.pivot(index = ['UNID','EXTERNAL_ID','QUESTION_ID'], columns = ['QUESTIONSET_ID'], values=['VALUE'])
            df_test.columns = df_test.columns.droplevel(0)
            df_test = df_test.rename_axis(None, axis = 1).reset_index()
            df_test = df_test.drop(columns = ['UNID'])
            
            df_test = dfQA.sort_values(by=['QUESTION_ID'])
        
            dfQA['QUESTION_ID'] = dfQA['QUESTION_ID'].str.lower()
            
            fieldlist = ['addr_type_cd', 'hiv_sex_male',
                         'addr_original_type_cd', 'address_dt', 'country', 'county','assigned_sex', 'birth_date', 'country_of_birth',
                         'hiv_current_status', 'hiv_sex_female', 'hiv_sex_male',
                         'hiv_hemphilia', 'factor_type', 'clotting_factor_date_received', 'sexbi','hetero_contact_transfusion_with_hiv', 
                         'hetero_contact_transplant_with_hiv', 'hetero_contact_with_hiv_not_specified', 
                         'hiv_blood_other','hiv_facility_city','hiv_blood_other_first_date', 'hiv_date_of_death', 'hiv_state_of_death',
                         'hiv_blood_other_last_date','hiv_transplant', 'hlth_worker','hiv_healthcare_setting_specify', 'hiv_healthcare_setting', 
                         'other_document_risk', 'nir','nir_dt', 'refer_obgyn','hiv_sex_injection_drugs_12mo_yes','hetero_contact_hemophilia_with_hiv']

            dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_healthcare_setting_specify'), 'QUESTION_ID'] = 'occupation'
            dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_healthcare_setting'), 'QUESTION_ID'] = 'hlth_worker'
            dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_blood_other_first_date'), 'QUESTION_ID'] = 'first_trans_date'
            dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_blood_other_last_date'), 'QUESTION_ID'] = 'last_trans_date'
            dfQA.loc[dfQA['QUESTION_ID'].str.contains('country_of_birth'), 'QUESTION_ID'] = 'birth_cntry'
            dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_sex_injection_drugs_yes'), 'QUESTION_ID'] = 'hiv_sex_injection_drugs'
            dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_sex_injection_drugs_unknown'), 'QUESTION_ID'] = 'hiv_sex_injection_drugs'
            dfQA.loc[dfQA['QUESTION_ID'] == 'sex_idu_yes', 'QUESTION_ID'] = 'sex_idu'
            dfQA.loc[dfQA['QUESTION_ID'] == 'sex_idu_unknown', 'QUESTION_ID'] = 'sex_idu'
            dfQA.loc[dfQA['QUESTION_ID'] == 'hiv_sex_male', 'QUESTION_ID'] = 'hiv_sex_male'
            dfQA.loc[dfQA['QUESTION_ID'] == 'hiv_sex_male_yes_12_yes', 'QUESTION_ID'] = 'hiv_sex_past_12_months_male'
            dfQA.loc[dfQA['QUESTION_ID'] == 'hiv_sex_female', 'QUESTION_ID'] = 'hiv_sex_female'
            dfQA.loc[dfQA['QUESTION_ID'] == 'hiv_sex_female', 'QUESTION_ID'] = 'hiv_sex_past_12_months_female'
            dfQA.loc[dfQA['QUESTION_ID'].str.contains('|'.join(fieldlist))]
            for vals in enumerate(fieldlist):
                dfQA.loc[dfQA['QUESTION_ID'].str.contains(vals[1]), 'QUESTION_ID'] = vals[1]
            
            dfQA.rename(columns={"country":"cntry", "county":"cnty", "assigned_sex":"birth_sex","hiv_current_status":"vital_status",
                                 "hiv_sex_female":"swf", "hiv_sex_injection_drugs":"idu","hiv_hemphilia":"rcvd_clot_factor", 
                                 "clotting_factor_date_received":"factor_dt", "sex_idu":"sexidu",
                                 "hetero_contact_hemophilia_with_hiv":"sexhemo","hetero_contact_transfusion_with_hiv":"sex_transf",
                                 "hetero_contact_transplant_with_hiv":"sex_transpl","hetero_contact_with_hiv_not_specified":"sex_aidshiv",
                                 "hiv_blood_other":"rcvd_transf","hiv_blood_other_first_date":"first_trans_dt","hiv_blood_other_last_date":"last_trans_dt",
                                 "hiv_transplant":"rcvd_transpl","hiv_healthcare_setting":"hlth_worker","other_document_risk":"other_risk",
                                 "hiv_facility_city":"facility_type", "hiv_sex_male":"swm"}, inplace = True)

            df_test = dfQA.pivot_table(index = ['EXTERNAL_ID'], columns = ['QUESTION_ID'], values=['VALUE'], aggfunc='sum')

            df_test.columns = df_test.columns.droplevel(0)
            df_test = df_test.rename_axis(None, axis = 1).reset_index()
            dfQA = dfQA.drop(columns = ['UNID','QUESTIONSET_ID','QUESTION_ID','VALUE']).drop_duplicates()

            dfQA = pd.merge(dfQA, df_test, how ='outer', on = 'EXTERNAL_ID')
            
            dfQA['record_no'] = np.arange(len(dfQA))+1
            rcount = dfQA.pop('record_no')
            dfQA.insert(0, rcount.name, rcount)
            dfQA['EXTERNAL_ID'] = dfQA['EXTERNAL_ID'].astype('str')
            dfQA['binary'] = dfQA['EXTERNAL_ID'].shift(1, fill_value = dfQA['EXTERNAL_ID'].head(1)) != dfQA['EXTERNAL_ID']
            dfQA['parent_id'] = dfQA['EXTERNAL_ID'].groupby((dfQA['binary']).cumsum()).cumcount()+1
            dfQA.loc[dfQA['parent_id'] == 1, 'rollup_no'] = dfQA['record_no']
            
            dfQA.rename(columns={"CASE_ID": "QUESTIONAIRE_CASE_ID", "EXTERNAL_ID": "CASE_ID"}, inplace = True)
            
            dfQA.set_index('record_no', inplace = True)
            dfQA = dfQA.drop(columns = ['binary'])
            
            col = dfQA.pop('parent_id')
            dfQA.insert(1, col.name, col)
            
            dfQA['parent_id'] = dfQA['parent_id'].astype(str)
            dfQA['CASE_ID'] = dfQA['CASE_ID'].astype(str)
            df['parent_id'] = df['parent_id'].astype(str)
            df['CASE_ID'] = df['CASE_ID'].astype(str)
            df1 = df[['CASE_ID','U_ID','INVUNID','parent_id','rollup_no','doc_type','STATENO','rcvd_dt','source','name_use_cd','f_name','m_name','l_name','birth_dt']]
            df1.columns = df1.columns.str.lower()
            df1 = df1.sort_values(by=['invunid'])
            dfQA.columns = dfQA.columns.str.lower()
            #dfQA = dfQA.sort_values(by=['INVUNID'])
            dfQA = pd.merge(df1, dfQA, how = 'outer', on=['case_id', 'parent_id']).reset_index(drop=True)
            dfQA = dfQA.drop(columns = 'rollup_no_x')
            
            dfQA['country_usd'] = ''
            dfQA['addr_original_type_cd'] = ''
            dfQA['nir_dt'] = ''
            dfQA['nir'] = 'UNKNOWN'
            #???
            dfQA['address_dt'] = ''
            dfQA['sexbi'] = ''
            dfQA['birth_usd'] = ''
            dfQA['cntry_usd'] = ''
            dfQA['addr_type_cd'] = ''
            dfQA['refer_obgyn'] = ''
            dfQA['factor_type'] = ''
            dfQA['doctype'] = '004'
            
            dfQA['invunid'] = dfQA['invunid'].astype('float')
            
            dfQA = dfQA.sort_values(by=['invunid'])
            
            dfQA['record_no'] = np.arange(len(dfQA))+1
            rcount = dfQA.pop('record_no')
            dfQA.insert(0, rcount.name, rcount)
            
            dfQA['record_no'] = dfQA['record_no'].astype('str')
            dfQA['invunid'] = dfQA['invunid'].astype('str') 
            dfQA.loc[dfQA['invunid'].isna(), 'invunid'] = dfQA['record_no']

            dfQA['binary'] = dfQA['invunid'].shift(1, fill_value = dfQA['invunid'].head(1)) != dfQA['invunid']
            dfQA['parent_id'] = dfQA['invunid'].groupby((dfQA['binary']).cumsum()).cumcount()+1
            dfQA['U_ID'] = dfQA.groupby('invunid', sort=False).ngroup()+1
            dfQA['rollup_no'] = np.nan
            dfQA.loc[dfQA['parent_id'] == 1, 'rollup_no'] = dfQA['record_no']
            dfQA['rollup_no'] = dfQA['rollup_no'].ffill()
            dfQA['rollup_no'] = dfQA['rollup_no'].astype('int')
            
            dfQA = dfQA.drop(columns = ['rollup_no_y','binary'])
            
            col = dfQA.pop('invunid')
            dfQA.insert(0, col.name, col)
            
            col = dfQA.pop('parent_id')
            dfQA.insert(1, col.name, col)
            
            col = dfQA.pop('U_ID')
            dfQA.insert(2, col.name, col)
            
            dfQA = dfQA.rename(columns=str.lower)
            
            col = dfQA.pop('rollup_no')
            dfQA.insert(3, col.name, col)
            
            col = dfQA.pop('doc_type')
            dfQA.insert(4, col.name, col)
            
            dfQA = dfQA.rename(columns={"country":"cntry", "state_x":"state", "zip":"zip_code", "county":"cnty", 
                                        "assigned_sex":"birth_sex", "country_of_birth":"birth_cntry","hiv_current_status":"vital_status",
                                        "hiv_sex_female":"swf", "hiv_sex_injection_drugs":"idu","hiv_hemphilia":"rcvd_clot_factor",
                                        "clotting_factor_date_received":"factor_dt",
                                        "sex_idu":"sexidu", "hetero_contact_transfusion_with_hiv":"sex_transf",
                                        "hetero_contact_hemophilia_with_hiv":"sexhemo",
                                        "hetero_contact_transplant_with_hiv":"sex_transpl","hetero_contact_with_hiv_not_specified":"sex_aidshiv",
                                        "hiv_blood_other":"rcvd_transf","hiv_blood_other_first_date":"first_trans_dt","hiv_blood_other_last_date":"last_trans_dt",
                                        "hiv_transplant":"rcvd_transpl","hiv_healthcare_setting":"hlth_worker","other_document_risk":"other_risk","hiv_sex_male":"swm"
                                        })
            
            dfQA = dfQA[['record_no','invunid','parent_id','rollup_no','doc_type','stateno','name_use_cd','f_name','m_name',
                         'l_name','addr_type_cd', 'addr_original_type_cd', 'address_dt', 'state', 'cntry', 'cntry_usd',
                         'birth_sex', 'birth_dt','birth_cntry', 'birth_usd', 'vital_status', 'swm',
                         'swf', 'idu', 'rcvd_clot_factor', 'factor_type', 'factor_dt', 'sexidu', 'sexbi', 'sex_transf',
                         'sexhemo','sex_transpl', 'sex_aidshiv', 'rcvd_transf',  'rcvd_transpl', 'hlth_worker', 'occupation',
                         'other_risk', 'nir', 'nir_dt']]
            
            dfQA.set_index('record_no', inplace = True)
            
            display(dfQA)
            
            return dfQA

    def Update_Last_Extract():
        cur = con.cursor()
        SQL = f'{eHSQL.SQLDateUpdate}' + recordlist
        cur.execute(SQL)
        global df2
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            df2 = pd.read_sql(SQL, con)
            
        display(df2)
        
        cur.close()
        con.commit()
        return df2

    open_connection()
    conn = True
    connection_test()
    close_connection()
    conn = False
    
    open_connection()
    conn = True
    records = False
    if tablecount(con) == False:
        print("No New Records.\n", end = "\r")
    else:
        print("\nRecords ready for processing.\n", end = "\r")
        
        date = datetime.today().strftime('%d%b%y_%H.%M.%S')
        
        tablecontent(con)
        
        get_data(con, recordlist)
        get_QandA(con, recordlist)

        
        #display(df)
        #display(dfQA)
        #
        #df = df[['U_ID',
        #         'INVUNID',
        #         'CASE_ID',
        #         'parent_id',
        #         'rollup_no',
        #         'doc_type',
        #         'STATENO',
        #         'rcvd_dt',
        #         'source',
        #         'surv_method',
        #         'name_use_cd',
        #         'f_name',
        #         'm_name',
        #         'l_name',
        #         'birth_dt',
        #         'facility_name',
        #         'facility_type',
        #         'facility_city',
        #         'facility_st',
        #         'facility_ctry',
        #         'facility_usd',
        #         'facility_phone',
        #         'lab_clia_no',
        #         'specimen_id',
        #         'accession_no',
        #         'colldate',
        #         'result_dt',
        #         'specimen_recieve_dt',
        #         'lab_test_name',
        #         'specimentype',
        #         'lab_result',
        #         'lab_units',
        #         'interpretation',
        #         'ResultValueTemp',
        #         'result_range_lower',
        #         'result_range_upper',
        #         'genotype_sequence',
        #         'starhs_sample_id',
        #         'sreason',
        #         'lab_specific_note',
        #         'comments']]
        
        #df.columns = df.columns.str.lower()
        #
        #dfQA_reduced = dfQA.copy()
        #        
        #dfQA_reduced['invunid'] = dfQA_reduced['invunid'].astype('float')
        #dfQA_reduced = dfQA_reduced[dfQA_reduced['invunid'].notna()]
        #dfQA_reduced['invunid'] = dfQA_reduced['invunid'].astype('int')
        #
        #dfQA_reduced = dfQA_reduced.drop(columns = ['parent_id','rollup_no','doc_type','stateno','name_use_cd','f_name','m_name','l_name'])
        #dfLRQA_reduced = pd.concat([df, dfQA_reduced.set_index(df.index)], axis=1)
        #display(dfLRQA_reduced)
        #
        #df = df.reset_index(drop = True)
        #dfQA = dfQA.reset_index(drop = True)
        #df = df.drop(columns = ['parent_id','rollup_no','doc_type','stateno','name_use_cd','f_name','m_name','l_name'])
        #dfLRQA = pd.concat([df, dfQA], axis=1)
        #dfLRQA['u_id'] = dfLRQA['u_id'].astype('float')
        #dfLRQA = dfLRQA.reset_index(drop = True)
        #dfLRQA['record_no'] = np.arange(len(dfLRQA))+1
        #rcount = dfLRQA.pop('record_no')
        #dfLRQA.insert(0, rcount.name, rcount)
        #dfLRQA.set_index('record_no', inplace = True)
        #display(dfLRQA)
        
        
        #df.to_csv(f'eHARSLabResultsOut_{str(date)}.csv', sep = '\t', encoding = 'utf-8', header = True)
        #dfQA.to_csv(f'eHARSLabQAOut_{str(date)}.csv', sep = '\t', encoding = 'utf-8', header = True)
        
        #display(df)
        #display(dfQA)
        
        #df = df.drop(columns = ['ResultValueTemp',
        #                            'INVUNID',
        #                            'INVRESULTUNID',
        #                            'CREATE_DATE', 
        #                            'MODIFICATION_DATE', 
        #                            'ReferenceRange', 
        #                            'Result', 
        #                            'ResultStatus', 
        #                            'ResultUnits', 
        #                            'TestLocalCode', 
        #                            'TestLocalDescription', 
        #                            'TestValue'])
            
        

        
        print("Deuteronomy 30:7","Genesis 12:3")
        #Update_Last_Extract()
        #GetQuestionSetUNID()
        #GetGlobalUNID()
        #for cases in caselist
        #get_data()en

except Exception as e:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    print(e, 'line: ', exc_tb.tb_lineno,'\n\n', traceback.format_exc())
    pass

except (Exception, O.Error) as dberr:
    print(dberr)
    logging.exception("message")
    pass

finally:
    logging.info(f'\nLOG END: {datetime.now()}\n')
    if conn == True:
        close_connection()

Connection established to:  Oracle Database 12c EE Extreme Perf Release 12.2.0.1.0 - 64bit Production

6 Existing Records

Records ready for processing.


,CASE_ID,UNID
0,100000571,181062204
1,100000008,2921245
2,100000296,90866775
3,100000381,122928347
4,100000518,166582986
5,100006970,574151293


('100000571', '100000008', '100000296', '100000381', '100000518', '100006970')


,STATENO,U_ID,parent_id,rollup_no,INVRESULTUNID,CREATE_DATE,MODIFICATION_DATE,FIRST_NAME,LAST_NAME,BIRTH_DATE,EXTENDED_VALUE,INVUNID,CASE_ID,Comments,ReferenceRange,Result,ResultStatus_x,ResultUnits,ResultValue,Test,TestLocalCode,TestLocalDescription,TestValue,CLIA,Facility,Phone,ResultStatus_y,ResultsChangeDate,SpecimenDate,SpecimenNumber,SpecimenReceivedDate,SpecimenSource,Notes,doc_type,source,surv_method,enter_dt,rcvd_dt,m_name,f_name,l_name,birth_dt,name_use_cd,isGenoType,specimen_id,lab_specific_note,comments,notes,addr_type_cd,facility_name,facility_ctry,facility_phone,refer_obgyn,lab_clia_no,accession_no,colldate,result_dt,specimen_recieve_dt,specimentype,lab_result,lab_units,lab_test_name,genotype_sequence,starhs_sample_id,sreason,ResultValueTemp,interpretation,result_range_upper,result_range_lower
0,201801,1,1,1,2921313,02/01/2018,02/01/2018,Layla,Bunny,01/05/1985,None,2921321,100000008,NaN,NaN,G-A385,NaN,NaN,NaN,14092-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01/17/2018,NaN,NaN,276014004,NaN,004,A05.03,P,02/01/2018,02/01/2018,,Layla,Bunny,01/05/1985,,False,NaN,NaN,NaN,NaN,,NaN,USA,NaN,,NaN,NaN,01/17/2018,01/17/2018,NaN,276014004,NaN,,14092-1,,,,NaN,NaN,10000000,20
1,201901,2,1,2,1067036732,09/15/2022,09/15/2022,Fred,Flintstone,01/11/2000,None,1067036744,100000296,NaN,30-56,ND000009,Final Results,%,35,8123-2,3319,% CD4 (T Cells),8123-2; CD3+CD4+ Cells NFr Bld,24D0404292,NaN,NaN,Final,09/13/2022,09/12/2022,I012029125,09/13/2022,53130003,NaN,004,A05.03,P,09/15/2022,09/15/2022,,Fred,Flintstone,01/11/2000,,False,I012029125,NaN,NaN,NaN,,NaN,USA,NaN,,24D0404292,I012029125,09/12/2022,09/12/2022,09/13/2022,53130003,35,PCT,32516-7,,,,35,<,10000000,20
2,201901,2,1,5,1067036709,09/15/2022,09/15/2022,Fred,Flintstone,01/11/2000,None,1067036744,100000296,NaN,497-2267,ND000009,Final Results,cells/mcL,730,24467-3,3325,CD4 (T Cells),24467-3; CD3+CD4+ Cells # Bld,24D0404292,NaN,NaN,Final,09/13/2022,09/12/2022,I012029125,09/13/2022,53130003,NaN,004,A05.03,P,09/15/2022,09/15/2022,,Fred,Flintstone,01/11/2000,,False,I012029125,NaN,NaN,NaN,,NaN,USA,NaN,,24D0404292,I012029125,09/12/2022,09/12/2022,09/13/2022,53130003,730,CNT,24467,,,,730,<,10000000,20
3,201901,3,1,3,1067036788,09/15/2022,09/15/2022,Fred,Flintstone,01/11/2000,None,1067036789,100000296,NaN,365-1437,ND000009,Final Results,cells/mcL,133,24467-3,3325,CD4 (T Cells),24467-3; CD3+CD4+ Cells # Bld,24D0404292,NaN,NaN,Final,09/13/2022,09/12/2022,I012029135,09/13/2022,53130003,NaN,004,A05.03,P,09/15/2022,09/15/2022,,Fred,Flintstone,01/11/2000,,False,I012029135,NaN,NaN,NaN,,NaN,USA,NaN,,24D0404292,I012029135,09/12/2022,09/12/2022,09/13/2022,53130003,133,CNT,24467,,,,133,<,10000000,20
4,201901,3,2,3,1067036778,09/15/2022,09/15/2022,Fred,Flintstone,01/11/2000,None,1067036789,100000296,NaN,32-64,ND000009,Final Results,%,13,8123-2,3319,% CD4 (T Cells),8123-2; CD3+CD4+ Cells NFr Bld,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,004,A05.03,P,09/15/2022,09/15/2022,,NaN,NaN,NaN,,False,NaN,NaN,NaN,NaN,,NaN,USA,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,13,PCT,32516-7,,,,13,>,10000000,20
5,120594,4,1,6,1066979840,09/15/2022,09/15/2022,Contact_test,Contact_test2,07/01/2009,None,1066979847,100000571,Interpretation of results: SUSC= Susceptible P...,NaN,ND000024,Final Results,NaN,,34700-5,37216,"HIV-1 Genotypic PR-RT Drug Resistance, P",34700-5; HIV RT+PR Mut Tested Islt,24D0404292,NaN,NaN,Final,02/03/2022,01/31/2022,H231045894,02/01/2022,50863008,Interpretation of results: SUSC= Susceptible P...,004,A05.03,P,09/15/2022,09/15/2022,,Contact_test,Contact_test2,07/01/2009,,True,H231045894,Interpretation of results: SUSC= Susceptible P...,Interpretation of results: SUSC= Susceptible P...,Interpretation of results: SUSC= Susceptible P...,,NaN,USA,NaN,,24D0404292,H231045894,01/31/2022,01/31/2022,02/01/2022,50863008,,BP,34700-5,None,,,,<,10000000,20
6,120594,5,1,7,1066930018,09/14/2022,09/14/2022,Contact_test,Contact_test2,07/01/2009,None,1066930024,100000571,SPECIMEN SOURCE: BLOOD SPECIMEN SPECIMEN SITE:...,Reference Value: Not Dete

,invunid,parent_id,rollup_no,doc_type,stateno,name_use_cd,f_name,m_name,l_name,addr_type_cd,addr_original_type_cd,address_dt,state,cntry,cntry_usd,birth_sex,birth_dt,birth_cntry,birth_usd,vital_status,swm,swf,idu,rcvd_clot_factor,factor_type,factor_dt,sexidu,sexbi,sex_transf,sexhemo,sex_transpl,sex_aidshiv,rcvd_transf,rcvd_transpl,hlth_worker,occupation,other_risk,nir,nir_dt
record_no,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2921321.0,1,1,004,201801,,Layla,,Bunny,,,,ND,USA,,NaN,01/05/1985,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNKNOWN,
2,992782688.0,1,2,004,2022001,,Daffy,,Duck,,,,ND,USA,,NaN,01/01/1981,NaN,,NaN,YES,NaN,NaN,NaN,,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNKNOWN,
3,1066930024.0,1,3,004,120594,,Contact_test,,Contact_test2,,,,ND,USA,,FEMALE,07/01/2009,USA,,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,,NaN,UNKNOWN,,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,TEST6110,UNKNOWN,UNKNOWN,
4,1066930024.0,2,3,004,120594,,NaN,,NaN,,,,NaN,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNKNOWN,
5,1066979847.0,1,5,004,120594,,Contact_test,,Contact_test2,,,,ND,USA,,FEMALE,07/01/2009,USA,,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,,NaN,UNKNOWN,,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,TEST6110,UNKNOWN,UNKNOWN,
6,1066980171.0,1,6,004,120594,,Contact_test,,Contact_test2,,,,ND,USA,,FEMALE,07/01/2009,USA,,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,,NaN,UNKNOWN,,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,TEST6110,UNKNOWN,UNKNOWN,
7,1066980171.0,2,6,004,120594,,NaN,,NaN,,,,NaN,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNKNOWN,
8,1067036744.0,1,8,004,201901,,Fred,,Flintstone,,,,ND,USA,,NaN,01/11/2000,NaN,,NaN,YES,NaN,NaN,YES,,01/01/1990,NaN,,YES,NO,NO,YES,NO,YES,NO,TEST,YES,UNKNOWN,
9,1067036744.0,2,8,004,201901,,Fred,,Flintstone,,,,ND,USA,,NaN,01/11/2000,NaN,,NaN,YES,NaN,NaN,YES,,01/01/1990,NaN,,YES,NO,NO,YES,NO,YES,NO,TEST,YES,UNKNOWN,


Deuteronomy 30:7 Genesis 12:3
